# OpenCV — v4 Webcam (Mosaïque complète de tous les filtres)

Cette version **v4** affiche **tous les filtres simultanément** en mosaïque (webcam en temps réel).
- **[ / ]** : diminuer / augmenter l’échelle (downscale) de traitement
- **S** : sauvegarder une image de la mosaïque
- **Q** : quitter
- L’overlay affiche le **FPS** et les dimensions de traitement.

> Remarque : pour rester fluide, certains filtres très coûteux peuvent être recalculés **toutes les N frames** (avec cache entre-temps).


## 0) Vérification de l’environnement

In [ ]:
import sys, platform, cv2, numpy as np
print("Python:", sys.executable)
print("Version Python:", platform.python_version())
print("OpenCV:", cv2.__version__)


Python: c:\Users\vassi\Desktop\livre opencv\.venv\Scripts\python.exe
Version Python: 3.11.0
OpenCV: 4.12.0


## 1) Définition des filtres (identiques à la v3)

In [ ]:
import cv2, numpy as np

def filt_identity(frame, **p):  # 1
    return frame

def filt_box_blur(frame, k=5, **p):  # 2
    k = max(1, int(k) | 1)
    return cv2.blur(frame, (k,k))

def filt_gauss_blur(frame, k=5, **p):  # 3
    k = max(1, int(k) | 1)
    return cv2.GaussianBlur(frame, (k,k), 0)

def filt_median(frame, k=5, **p):  # 4
    k = max(1, int(k) | 1)
    return cv2.medianBlur(frame, k)

def filt_bilateral(frame, d=9, sc=75, ss=75, **p):  # 5
    return cv2.bilateralFilter(frame, int(d), float(sc), float(ss))

def filt_sobel_mag(frame, k=3, **p):  # 6
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=int(k))
    sy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=int(k))
    mag = cv2.magnitude(sx, sy)
    mag = np.clip(mag, 0, 255).astype(np.uint8)
    return cv2.cvtColor(mag, cv2.COLOR_GRAY2BGR)

def filt_laplacian(frame, k=3, **p):  # 7
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lap = cv2.Laplacian(gray, cv2.CV_16S, ksize=int(k))
    lap = cv2.convertScaleAbs(lap)
    return cv2.cvtColor(lap, cv2.COLOR_GRAY2BGR)

def filt_canny(frame, t1=100, t2=200, **p):  # 8
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, int(t1), int(t2))
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

def filt_unsharp(frame, k=5, amount=1.5, **p):  # 9
    k = max(1, int(k) | 1)
    blur = cv2.GaussianBlur(frame, (k,k), 0)
    sharp = cv2.addWeighted(frame, 1+float(amount), blur, -float(amount), 0)
    return sharp

def filt_emboss(frame, **p):  # 10
    kernel = np.array([[-2,-1,0],
                       [-1, 1,1],
                       [ 0, 1,2]], dtype=np.float32)
    out = cv2.filter2D(frame, -1, kernel) + 128
    return np.clip(out, 0, 255).astype(np.uint8)

def filt_clahe(frame, clip=2.0, grid=8, **p):  # 11
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=float(clip), tileGridSize=(int(grid), int(grid)))
    l2 = clahe.apply(l)
    lab2 = cv2.merge([l2,a,b])
    return cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

def filt_thresholds(frame, t=128, mode=0, **p):  # 12 global / Otsu / adaptatif
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mode=int(mode)
    if mode==0:  # global
        _,th = cv2.threshold(gray, int(t), 255, cv2.THRESH_BINARY)
    elif mode==1:  # Otsu
        _,th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    else:  # adaptatif
        th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    return cv2.cvtColor(th, cv2.COLOR_GRAY2BGR)

def filt_posterize(frame, levels=4, **p):  # 13
    levels = max(2, int(levels))
    step = 256//levels
    return (frame//step)*step + step//2

def filt_negative(frame, **p):  # 14
    return 255 - frame

def filt_sepia(frame, **p):  # 15
    M = np.array([[0.272,0.534,0.131],
                  [0.349,0.686,0.168],
                  [0.393,0.769,0.189]])
    sep = cv2.transform(frame, M.astype(np.float32))
    return np.clip(sep,0,255).astype(np.uint8)

def filt_dominant_channel(frame, **p):  # 16 argmax canal
    b,g,r = cv2.split(frame)
    idx = np.argmax(np.stack([b,g,r], axis=-1), axis=-1).astype(np.uint8)
    palette = np.array([[255,0,0],[0,255,0],[0,0,255]], dtype=np.uint8)  # B,G,R
    out = palette[idx]
    return out

def filt_erode(frame, k=3, it=1, **p):  # 17
    k = max(1, int(k) | 1)
    ker = cv2.getStructuringElement(cv2.MORPH_RECT, (k,k))
    return cv2.erode(frame, ker, iterations=int(it))

def filt_dilate(frame, k=3, it=1, **p):  # 18
    k = max(1, int(k) | 1)
    ker = cv2.getStructuringElement(cv2.MORPH_RECT, (k,k))
    return cv2.dilate(frame, ker, iterations=int(it))


## 2) Boucle webcam + mosaïque complète

In [ ]:
import cv2, numpy as np, time, math, os

FILTERS = [
    ("Original",            filt_identity),
    ("Box Blur",            filt_box_blur),
    ("Gaussian Blur",       filt_gauss_blur),
    ("Median Blur",         filt_median),
    ("Bilateral",           filt_bilateral),
    ("Sobel Magnitude",     filt_sobel_mag),
    ("Laplacian",           filt_laplacian),
    ("Canny",               filt_canny),
    ("Unsharp Mask",        filt_unsharp),
    ("Emboss",              filt_emboss),
    ("CLAHE",               filt_clahe),
    ("Thresholds",          filt_thresholds),
    ("Posterize",           filt_posterize),
    ("Negative",            filt_negative),
    ("Sepia",               filt_sepia),
    ("Dominant Channel",    filt_dominant_channel),
    ("Erode",               filt_erode),
    ("Dilate",              filt_dilate),
]

# Paramètres par filtre (valeurs par défaut raisonnables)
PARAMS = {
    "Box Blur":          {"k":5},
    "Gaussian Blur":     {"k":5},
    "Median Blur":       {"k":5},
    "Bilateral":         {"d":9, "sc":75, "ss":75},
    "Sobel Magnitude":   {"k":3},
    "Laplacian":         {"k":3},
    "Canny":             {"t1":100, "t2":200},
    "Unsharp Mask":      {"k":5, "amount":1.5},
    "CLAHE":             {"clip":2.0, "grid":8},
    "Thresholds":        {"t":128, "mode":0},  # 0:global,1:Otsu,2:Adapt
    "Posterize":         {"levels":4},
    "Erode":             {"k":3, "it":1},
    "Dilate":            {"k":3, "it":1},
}

# Fréquence de recalcul (1 = chaque frame). On ménage "Bilateral".
RECALC_EVERY = {
    "Bilateral": 3
}

def overlay_name(img, name):
    # Bande en bas du tile, pour éviter tout chevauchement avec le bandeau global du haut
    h, w = img.shape[:2]
    cv2.rectangle(img, (0, h-24), (w, h), (0,0,0), -1)
    cv2.putText(img, name, (6, h-7), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,255,0), 1, cv2.LINE_AA)
    return img

def tile_grid(images, cols=6):
    # images: list of (name, bgr)
    tiles = []
    for name, im in images:
        tiles.append(overlay_name(im.copy(), name))
    rows = math.ceil(len(tiles)/cols)
    grid_rows = []
    idx = 0
    w = tiles[0].shape[1]
    h = tiles[0].shape[0]
    blank = np.zeros((h,w,3), dtype=np.uint8)
    for r in range(rows):
        row_imgs = []
        for c in range(cols):
            if idx < len(tiles):
                row_imgs.append(tiles[idx])
            else:
                row_imgs.append(blank.copy())
            idx += 1
        grid_rows.append(np.hstack(row_imgs))
    mosaic = np.vstack(grid_rows)
    return mosaic

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
assert cap.isOpened(), "Webcam non détectée."

win = "OpenCV v4 — Mosaïque complète"
cv2.namedWindow(win, cv2.WINDOW_NORMAL)

scale = 0.5  # downscale de traitement (0.2–1.0)
cols = 6     # nb colonnes pour la mosaïque
frame_id = 0
prev = time.time()
fps = 0.0

# cache pour filtres recalculés moins souvent
cache = {name: None for name,_ in FILTERS}
cache_frame = {name: -999 for name,_ in FILTERS}

while True:
    ok, frame = cap.read()
    if not ok:
        break
    frame = cv2.flip(frame, 1)
    frame_id += 1

    # downscale pour le calcul des filtres
    th = int(frame.shape[0]*scale)
    tw = int(frame.shape[1]*scale)
    small = cv2.resize(frame, (tw, th), interpolation=cv2.INTER_AREA)

    # appliquer tous les filtres (avec skip sur certains)
    tiles = []
    for name, fn in FILTERS:
        recalc_every = RECALC_EVERY.get(name, 1)
        if (frame_id - cache_frame[name]) >= recalc_every:
            params = PARAMS.get(name, {})
            out = fn(small, **params)
            if out.ndim == 2:
                out = cv2.cvtColor(out, cv2.COLOR_GRAY2BGR)
            # assurer même taille
            out = cv2.resize(out, (tw, th), interpolation=cv2.INTER_AREA)
            cache[name] = out
            cache_frame[name] = frame_id
        tiles.append((name, cache[name]))

    mosaic = tile_grid(tiles, cols=cols)

    # FPS
    now = time.time()
    dt = now - prev
    if dt > 0:
        fps = 0.9*fps + 0.1*(1.0/dt)
    prev = now

    # Overlay global
    info = f"[{int(scale*100)}%] {tw}x{th} — {len(FILTERS)} filtres — {fps:.1f} FPS — [/] scale | S save | Q quit"
    cv2.rectangle(mosaic, (0,0), (mosaic.shape[1], 28), (40,40,40), -1)
    cv2.putText(mosaic, info, (8,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 1, cv2.LINE_AA)

    cv2.imshow(win, mosaic)
    k = cv2.waitKey(1) & 0xFF
    if k in (ord('q'), ord('Q')):
        break
    elif k == ord(']'):
        scale = min(1.0, scale + 0.1)
    elif k == ord('['):
        scale = max(0.2, scale - 0.1)
    elif k in (ord('s'), ord('S')):
        ts = time.strftime("%Y%m%d_%H%M%S")
        path = f"/mnt/data/mosaic_{ts}.png"
        cv2.imwrite(path, mosaic)
        print("Image mosaïque sauvegardée:", path)

cap.release()
cv2.destroyAllWindows()


Image mosaïque sauvegardée: /mnt/data/mosaic_20251108_090453.png
Image mosaïque sauvegardée: /mnt/data/mosaic_20251108_090457.png
